In [1]:
import torch
from torch.utils.data import Dataset,DataLoader
from torchvision import models,transforms


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [3]:
import json
import cv2
from cv2 import cvtColor,COLOR_BGR2RGB
import numpy as np

In [5]:
from cv2 import cvtColor,COLOR_BGR2RGB
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file,'r')as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229,0.224,0.225])

        ])
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
       item = self.data[idx]
       img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
       h,w = img.shape[:2]

       img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

       img = self.transforms(img)
       kps = np.array(item['kps']).flatten()
       kps = kps.astype(np.float32)

       kps[::2] *= 224.0/w
       kps[1::2] *= 224.0/h

       return img,kps


In [6]:
train_dataset = KeypointsDataset(r"C:\Users\ankit\Downloads\tennis_court_det_dataset\data\images", r"C:\Users\ankit\Downloads\tennis_court_det_dataset\data\data_train.json")
val_dataset = KeypointsDataset(r"C:\Users\ankit\Downloads\tennis_court_det_dataset\data\images", r"C:\Users\ankit\Downloads\tennis_court_det_dataset\data\data_val.json")
train_loader = DataLoader(train_dataset,batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=8,shuffle=True)

In [7]:
model = models.resnet50(pretrained = True)
model.fc = torch.nn.Linear(model.fc.in_features,14*2)

c:\Users\ankit\OneDrive\Desktop\Tennis\tennis.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ankit\OneDrive\Desktop\Tennis\tennis.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
model = model.to(device)

In [9]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)

In [10]:
epochs=2
for epoch in range(epochs):
    for i, (img,kps) in enumerate(train_loader):
        img = img.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(img)
        loss = criterion(outputs,kps)
        loss.backward()
        optimizer.step()

        if i%10 == 0:
            print(f"Epoch{epoch+1}/{epochs}, step {i+1}/{len(train_loader),}Loss:{loss.item()}")

Epoch1/2, step 1/(829,)Loss:14765.150390625
Epoch1/2, step 11/(829,)Loss:14457.443359375
Epoch1/2, step 21/(829,)Loss:13217.5322265625
Epoch1/2, step 31/(829,)Loss:13391.662109375
Epoch1/2, step 41/(829,)Loss:13920.8896484375
Epoch1/2, step 51/(829,)Loss:12575.0478515625
Epoch1/2, step 61/(829,)Loss:13334.6357421875
Epoch1/2, step 71/(829,)Loss:13253.0556640625
Epoch1/2, step 81/(829,)Loss:12018.9921875
Epoch1/2, step 91/(829,)Loss:11780.7314453125
Epoch1/2, step 101/(829,)Loss:10776.1650390625
Epoch1/2, step 111/(829,)Loss:11047.1611328125
Epoch1/2, step 121/(829,)Loss:10602.5390625
Epoch1/2, step 131/(829,)Loss:10306.9716796875
Epoch1/2, step 141/(829,)Loss:9954.6884765625
Epoch1/2, step 151/(829,)Loss:9652.5068359375
Epoch1/2, step 161/(829,)Loss:8904.2939453125
Epoch1/2, step 171/(829,)Loss:8641.1953125
Epoch1/2, step 181/(829,)Loss:8537.583984375
Epoch1/2, step 191/(829,)Loss:8206.2265625
Epoch1/2, step 201/(829,)Loss:7463.18505859375
Epoch1/2, step 211/(829,)Loss:7822.09814453125

In [13]:
torch.save(model.state_dict(),"keypoints_model.pth")